### Facultad de Ciencias Exactas, Ingeniería y Agrimensura - UNR
## Tenicatura Universitaria en Inteligencia Artificial.


# Trabajo Práctico Final: Procesamiento de Lenguaje Natural.

- **Pace, Bruno. Legajo: P-5295/7.**

Docentes:
 - Manson, Juan Pablo.
 - Geary, Alan.
 - Sollberger, Dolores.
 - Ferrucci, Costantino.

Fecha de Entrega: 15 de diciembre de 2024.


In [1]:
# Redis graph
%%capture
!pip install redis redisgraph
!wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!curl -fsSL https://packages.redis.io/redis-stack/redis-stack-server-6.2.6-v7.focal.x86_64.tar.gz -o redis-stack-server.tar.gz
!tar -xvf redis-stack-server.tar.gz
!./redis-stack-server-6.2.6-v7/bin/redis-stack-server --daemonize yes


import redis
from redisgraph import Graph, Node, Edge
import matplotlib.pyplot as plt
import networkx as nx

# Selenium web scrapping
!apt-get update
!apt-get install -y chromium-browser chromium-chromedriver
!pip install selenium

!apt-get install -y xvfb
!pip install pyvirtualdisplay

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

import os
os.environ["PATH"] += ":/usr/bin/chromedriver"


# Base de Datos de Grafo

## Web scrapping
Se obtiene información a través de Web Scrapping con Selenium.
Se tiene la siguiente información:
- Diseñadores.
- Artistas.
- Familia de Juegos.
- Mecanismos.
- Empresas que lo publicaron.
- Categorias.

In [2]:
url_grafo: str = "https://boardgamegeek.com/boardgame/220877/rajas-of-the-ganges/credits"
title: str = 'Rajas Of The Ganges'

In [3]:
def buscar_credito(data: str) -> tuple[dict[str, list[str]], dict[str,list[str]]]:
    """
    Retorna una tupla de diccionarios que contienen el(los) nombre(s) y el link(s) correspondiente(s)

    Args:
      data: str = el nombre del dato a buscar

    Returns:
      name_dict: dict[str, list[str]] = diccionario con los nombres.
      link_dict: dict[str, list[str]] = diccionario con los links.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(service=Service(), options=chrome_options)
    driver.get(url_grafo)

    name_dict: dict[str, list[str]] = {f"{data}_nombre": []}
    link_dict: dict[str, list[str]] = {f"{data}_link": []}

    elements = driver.find_elements(By.XPATH, f'//a[contains(@ng-href, "/boardgame{data}/")]')

    if elements:
        for element in elements:
            name_dict[f"{data}_nombre"].append(element.text)
            link_dict[f"{data}_link"].append(element.get_attribute('href'))
    else:
        print(f"No se encontraron elementos para {data}")

    driver.quit()

    return name_dict, link_dict

data_designer: tuple[dict[str, list[str]], dict[str,list[str]]] = buscar_credito("designer")
data_artist: tuple[dict[str, list[str]], dict[str,list[str]]] = buscar_credito("artist")
data_family: tuple[dict[str, list[str]], dict[str,list[str]]] = buscar_credito("family")
data_mechanic: tuple[dict[str, list[str]], dict[str,list[str]]] = buscar_credito("mechanic")
data_publisher: tuple[dict[str, list[str]], dict[str,list[str]]] = buscar_credito("publisher")
data_categories: tuple[dict[str, list[str]], dict[str,list[str]]] = buscar_credito("category")

In [4]:
data_categories[0]

{'category_nombre': ['Dice', 'Economic', 'Renaissance', 'Territory Building']}

## Creación de BBDD de Grafos
Se utiliza Redis, por su simple implementación con Colab.

In [5]:
redis_client = redis.Redis(host='localhost', port=6379)

In [6]:
def define_relation(label: str) -> str:
  """
  Define la relación entre el nodo titulo y el nodo que se quiere relacionar

  Args:
    label: str = el label del nodo que se quiere relacionar

  Returns:
    str = el label de la relación
  """
  if label == "Designer":
    return "Designed by"
  elif label == "Artist":
    return "Created by"
  elif label == "Family":
    return "Similar game"
  elif label == "Mechanic":
    return "Mechanic"
  elif label == "Published by":
    return "Published by"
  elif label == "Category":
    return "Category"
  return None

In [7]:
def crear_nodos_with_edges(graph: Graph, data: dict[str, list[str]], main_node: Node) -> None:
    """
    Dado un diccionario con datos, crea los nodos correspondientes en un grafo
    de redis y añade las relaciones correspondientes.

    Args:
        graph: Redis graph instance.
        data: Dictionary containing names as lists under specific keys.
        main_node: The main node to relate new nodes with.

    Returns:
        None
    """
    if not data:
        print("Diccionario vacío")
        return

    for key, names in data[0].items():
        label = key.split("_")[0].capitalize()
        relation = define_relation(label)

        if not relation:
            print(f"Relación no definida para el label: {label}")
            continue

        for name in names:
            print(f'Creando nodo con nombre: {name} y label: {label}')
            node = Node(label=label, properties={"name": name})
            graph.add_node(node)
            graph.commit()

            print(f'Añadiendo relación: {relation}')
            edge = Edge(main_node, relation, node)
            graph.add_edge(edge)
            graph.commit()

    print("Nodos y relaciones creados exitosamente.")
    return None

In [8]:
graph = Graph('creditos', redis_client)
main_node = Node(label='title', properties={"name": title})
graph.add_node(main_node)
graph.commit()

In [9]:
crear_nodos_with_edges(graph, data_designer, main_node)
crear_nodos_with_edges(graph, data_artist, main_node)
crear_nodos_with_edges(graph, data_family, main_node)
crear_nodos_with_edges(graph, data_mechanic, main_node)
crear_nodos_with_edges(graph, data_publisher, main_node)
crear_nodos_with_edges(graph, data_categories, main_node)

Creando nodo con nombre: Inka Brand y label: Designer
Añadiendo relación: Designed by


ResponseError: errMsg: Invalid input 'b': expected '|', '*', '{', a parameter or ']' line: 1, column: 154, offset: 153 errCtx: ...e:"Rajas Of The Ganges"})-[:Designed by]->(ghprwbaxln:Designer{name:"Inka ... errCtxOffset: 40

Creación de las relaciones

In [ ]:
#graph.add_edge(edge)
data_graph_list: list = [data_designer[0], data_artist[0], data_mechanic[0], data_publisher[0], data_categories[0], data_family[0]]


for data in data_graph_list:
    #print(data)
    for key, names in data.items():
      label: str = key.split("_")[0].capitalize()

      for name in names:
        relation = define_relation(label)
        if relation is not None:
          edges.append(Edge(node,relation,name))

for edge in edges:
  graph.add_edge(edge)
  graph.commit()


In [ ]:
def get_all_nodes(graph: Graph) -> list[dict]:
    """
    Retrieve all nodes from the Redis graph.

    Args:
        graph: Redis graph instance.

    Returns:
        List of dictionaries where each dictionary contains the node's label, id, and properties.
    """
    query = "MATCH (n) RETURN n"
    result = graph.query(query)
    nodes = []

    for record in result.result_set:
        node = record[0]
        node_data = {
            "id": node.id,
            "label": node.label,
            "properties": node.properties
        }
        nodes.append(node_data)

    return nodes

# Example usage
all_nodes = get_all_nodes(graph)
for node in all_nodes:
    print(node)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from redisgraph import Graph, Node


def plot_graph(graph: Graph) -> None:
    """
    Plot the Redis graph using NetworkX and Matplotlib, optimized for larger graphs.

    Args:
        graph: Redis graph instance.
    """
    nx_graph = nx.DiGraph()

    node_query = "MATCH (n) RETURN n"
    node_result = graph.query(node_query)

    for record in node_result.result_set:
        node = record[0]
        node_id = node.id
        node_label = node.label
        node_name = node.properties.get("name", f"Node {node_id}")
        nx_graph.add_node(node_id, label=node_label, name=node_name, category=node_label)

    edge_query = "MATCH (n)-[r]->(m) RETURN n, r, m"
    edge_result = graph.query(edge_query)

    for record in edge_result.result_set:
        src = record[0].id
        dest = record[2].id
        relationship = record[1].type
        nx_graph.add_edge(src, dest, label=relationship)

    plt.figure(figsize=(16, 12))
    pos = nx.spring_layout(nx_graph, k=0.5)

    categories = nx.get_node_attributes(nx_graph, 'category')
    unique_categories = set(categories.values())
    node_colors = {cat: plt.cm.tab10(i / len(unique_categories)) for i, cat in enumerate(unique_categories)}
    node_color_list = [node_colors[categories[node]] for node in nx_graph.nodes()]

    nx.draw(
        nx_graph, pos,
        with_labels=False,
        node_size=700,
        node_color=node_color_list,
        edge_color='gray',
        alpha=0.7
    )

    node_labels = {node: data['name'] for node, data in nx_graph.nodes(data=True)}
    nx.draw_networkx_labels(nx_graph, pos, labels=node_labels, font_size=8, font_color='black')

    edge_labels = nx.get_edge_attributes(nx_graph, "label")
    nx.draw_networkx_edge_labels(nx_graph, pos, edge_labels=edge_labels, font_size=6)

    legend_labels = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=cat)
                     for cat, color in node_colors.items()]
    plt.legend(handles=legend_labels, loc='best', title='Categories')

    plt.title("Redis Graph Visualization")
    plt.show()


plot_graph(graph)
